In [1]:
using Statistics
using MLJBase, CategoricalArrays

# We use flux only to get the MNIST
using Flux, Flux.Data.MNIST

#push!(LOAD_PATH, "../src/") ## Uncomment if MulticlassPerceptron not installed
using MulticlassPerceptron


┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /Users/davidbuchaca1/.julia/packages/CUDAdrv/3EzC1/src/CUDAdrv.jl:69
┌ Info: Precompiling MulticlassPerceptron [dab37cba-9818-490d-9918-279965c31300]
└ @ Base loading.jl:1273


## Loading the data



In [2]:
function load_MNIST( ;array_eltype::DataType=Float32, verbose::Bool=true)

    if verbose
        time_init = time()
        println("\nMNIST Dataset Loading...")
    end
    train_imgs = MNIST.images(:train)                             # size(train_imgs) -> (60000,)
    test_imgs  = MNIST.images(:test)                              # size(test_imgs)  -> (10000,)
    train_x    = array_eltype.(hcat(reshape.(train_imgs, :)...))  # size(train_x)    -> (784, 60000)
    test_x     = array_eltype.(hcat(reshape.(test_imgs, :)...))   # size(test_x)     -> (784, 60000)

    ## Prepare data
    train_y = MNIST.labels(:train) .+ 1;
    test_y  = MNIST.labels(:test)  .+ 1;

    ## Encode targets as CategoricalArray objects
    train_y = CategoricalArray(train_y)
    test_y  = CategoricalArray(test_y)

    if verbose
        time_taken = round(time()-time_init; digits=3)
        println("\nMNIST Dataset Loaded, it took $time_taken seconds")
    end
    return train_x, train_y, test_x, test_y
end

println("\nLoading data\n")
train_x, train_y, test_x, test_y = load_MNIST( ;array_eltype=Float32, verbose=true)



Loading data


MNIST Dataset Loading...

MNIST Dataset Loaded, it took 0.561 seconds


(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CategoricalValue{Int64,UInt32}[6, 1, 5, 2, 10, 3, 2, 4, 2, 5  …  10, 3, 10, 6, 2, 9, 4, 6, 7, 9], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CategoricalValue{Int64,UInt32}[8, 3, 2, 1, 5, 2, 5, 10, 6, 10  …  8, 9, 10, 1, 2, 3, 4, 5, 6, 7])

In [3]:
## Define model and train it
n_features = size(train_x, 1);
n_classes  = length(unique(train_y));
y = MNIST.labels(:train) .+ 1;

## MulticlassPerceptronCore Object

The simplest way to train a MulticlassPerceptron is using the `MulticlassPerceptronCore` struct that simply stores the basic information of the model.

```julia
mutable struct MulticlassPerceptronCore{T}
    W::AbstractMatrix{T}
    b::AbstractVector{T}
    n_classes::Int
    n_features::Int
    is_sparse::Bool
end
```

In [4]:
is_sparse = false
perceptron = MulticlassPerceptronCore(Float32, n_classes, n_features, is_sparse) 

MulticlassPerceptronCore{Float32}(Float32[0.23447037 0.4702673 … 0.07694912 0.52801335; 0.25174487 0.6456789 … 0.1692096 0.54663014; … ; 0.07425809 0.14686751 … 0.30431843 0.93198514; 0.6398387 0.5903474 … 0.23373556 0.53661954], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 10, 784, false)

In [5]:
fit!(perceptron,
     train_x,
     y;
     verbosity=1,
     n_epochs=50,
     f_average_weights=true)

Epoch: 50 	 Accuracy: 0.898

10-element Array{Float32,1}:
 -59.287125
  36.523853
  19.533762
 -21.265474
   5.193879
 101.60766 
 -33.41662 
  54.046825
 -89.69275 
 -13.244057

In [6]:
y_hat_train = MulticlassPerceptron.predict(perceptron, train_x)
y_hat_test  = MulticlassPerceptron.predict(perceptron, test_x);

## Evaluate the model
println("Results:")
println("Train accuracy:", mean(y_hat_train .== train_y))
println("Test accuracy:",  mean(y_hat_test  .== test_y))
println("\n")

Results:
Train accuracy:0.9357166666666666
Test accuracy:0.9266




## MulticlassPerceptronClassifier Object

The `MulticlassPerceptronClassifier` is the basic object build to take advantage of MLJ capabilities.

We can use `fit` with a CategoricalArray of the target (which can have any values, it is not restricted to values from 1 to the number of classes). 

In [7]:
## Define model and train it
n_features = size(train_x, 1);
n_classes  = length(unique(train_y));
perceptron = MulticlassPerceptronClassifier(n_epochs=50; f_average_weights=true)

MulticlassPerceptronClassifier(n_epochs = 50,
                               n_epoch_patience = 5,
                               f_average_weights = true,
                               f_shuffle_data = false,
                               element_type = Float32,) @ 1…08

Training a `MulticlassPerceptronClassifier`

In [8]:
## Train the model
println("\nStart Learning\n")
time_init = time()
fitresult, _ , _  = fit(perceptron, 1, train_x, train_y) #
time_taken = round(time()-time_init; digits=3)


Start Learning

Epoch: 50 	 Accuracy: 0.898

6.534

Make predictions

In [9]:
println("\nLearning took $time_taken seconds\n")

## Make predictions
y_hat_train = MLJBase.predict(fitresult, train_x)
y_hat_test  = MLJBase.predict(fitresult, test_x);

## Evaluate the model
println("Results:")
println("Train accuracy:", mean(y_hat_train .== train_y))
println("Test accuracy:",  mean(y_hat_test  .== test_y))
println("\n")


Learning took 6.534 seconds

Results:
Train accuracy:0.9357833333333333
Test accuracy:0.9266




## Machine with `MulticlassPerceptronClassifier`



In [10]:
using MLJ

In [11]:
size(train_x)

(784, 60000)

In [12]:
train_x_rowexamples = MLJBase.table(train_x')  
train_x_rowexamples

Tables.MatrixTable{LinearAlgebra.Adjoint{Float32,Array{Float32,2}}}(Symbol[:x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10  …  :x775, :x776, :x777, :x778, :x779, :x780, :x781, :x782, :x783, :x784], Dict(:x753 => 753,:x620 => 620,:x233 => 233,:x71 => 71,:x110 => 110,:x685 => 685,:x348 => 348,:x630 => 630,:x539 => 539,:x608 => 608…), Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [13]:
perceptron = MulticlassPerceptronClassifier(n_epochs=50; f_average_weights=true)

MulticlassPerceptronClassifier(n_epochs = 50,
                               n_epoch_patience = 5,
                               f_average_weights = true,
                               f_shuffle_data = false,
                               element_type = Float32,) @ 5…34

In [14]:
# machines expert Tables.Table or DataFrame objects, not AbstractArrays
perceptron_machine = machine(perceptron, train_x_rowexamples, train_y)  

Machine{MulticlassPerceptronClassifier} @ 1…17


In [15]:
## Train the model
println("\nStart Learning\n")
time_init = time()
#fitresult, _ , _  = MLJBase.fit(perceptron, 1, train_x, train_y) # If train_y is a CategoricalArray
MLJBase.fit!(perceptron_machine)
time_taken = round(time()-time_init; digits=3)
println("\nLearning took $time_taken seconds\n")


Start Learning



┌ Info: Training Machine{MulticlassPerceptronClassifier} @ 1…17.
└ @ MLJ /Users/davidbuchaca1/.julia/packages/MLJ/K371Q/src/machines.jl:141


Epoch: 50 	 Accuracy: 0.899
Learning took 11.513 seconds



In [16]:
## Make predictions
y_hat_train = MLJBase.predict(perceptron_machine, train_x)
y_hat_test  = MLJBase.predict(perceptron_machine, test_x);

## Evaluate the model
println("Results:")
println("Train accuracy:", mean(y_hat_train .== train_y))
println("Test accuracy:",  mean(y_hat_test  .== test_y))
println("\n")

Results:
Train accuracy:0.93605
Test accuracy:0.9271




### EnsembleModel

### Composing Models